# Sequence Homology Data Prepration<a id='top'></a>
**Sections:**<br>
[0) Description](#0)<br>
[1) BLAST](#1)<br>
[2) Importing Modules and Packages](#2)<br>
[3) Configuration](#3)<br>
[4) Loading Gene Ontology](#4)<br>
[5) Loading Genes and Annotations](#5)<br>
[6) Loading Sequence Homology data](#6)<br>
[7) Computing log-reciprocal BLAST score (LRBS) and relative reciprocal BLAST score (RRBS)](#7)<br>
[8) Saving the results](#8)<br>


## Description<a id='0'></a>

**Aim:** This jupyter notebook results in **protein sequence similarity** data of species of interest (e.g., _human_ or *yeast*) with which the deepSimDEF networks would be trained and evaluatied. The similarity metrics employed are LRBS and RRBS.

---
**Output file format:** (space separated)<br>
_Protein_1_ _Protein_2_ _LRBS_ _RRBS_ <br>
A6NMZ5 A6NND4 2.095169 0.195754<br>
O94885 Q96HU1 1.606381 0.019799<br>
P29320 Q9Y572 1.983626 0.061849<br>
Q8IY84 Q8TEA7 1.694605 0.036144<br>
...<br>

---
Files needed for this preprocessing are:

 * **Gene ontology:** ['go.obo' file](http://current.geneontology.org/ontology/go.obo)<br><br>
 
 * **Association files:** [gene association files ingested from GO Consortium members](http://current.geneontology.org/products/pages/downloads.html)
  * **Human** - [Gene Association file (Homo sapiens)](http://geneontology.org/gene-associations/goa_human.gaf.gz)
  * **Yeast** - [Gene Association file (Saccharomyces cerevisiae)](http://current.geneontology.org/annotations/sgd.gaf.gz)<br><br>
  
 * **FASTA data:** [UniProt website (Proteomes)](https://www.uniprot.org/proteomes/)<br>
         
     * **Human** - [Proteomes - Homo sapiens (Human)](https://www.uniprot.org/proteomes/UP000005640)
         * Protein sequences: [one protein sequence per gene (FASTA)](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640_9606.fasta.gz)
         
     * **Yeast** - [Proteomes - Saccharomyces cerevisiae (Baker's yeast)](https://www.uniprot.org/proteomes/UP000002311)
         * Protein sequences: [one protein sequence per gene (FASTA)](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000002311_559292.fasta.gz)
         
[back to top](#top)<br>

## BLAST<a id='1'></a>
To get the sequence similarity from BLAST I used Compute Canada server on which blast was installed in advance; if you do not use that servers, make sure you have BLAST installed on you system . The results from blast (<code>blastp</code> for protein similarity) will be what we process here. The steps to get the <code>blastp</code> result are as follow.<br>

Keep in mind the we work with *E-value* of $0.1$ (by setting <code>-evalue 10</code>). For more information refer to [What is the Expect (E) value?](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=FAQ#expect).

---
#### load blast module
<font size="2">
For Compute Canada servers you need to load these (the <code>nixpkgs</code> and <code>gcc</code> are requisite for <code>blast+</code>):<br>
<code>module load nixpkgs/16.09 gcc/7.3.0 blast+/2.10.0</code><br>
Otherwise install BLAST from this link: [Basic Local Alignment Search Tool](https://blast.ncbi.nlm.nih.gov/Blast.cgi)
</font>

---
#### human sequence (FASTA)<br>
<font size="2">
1) Download the protein sequences (human) in zipped FASTA format (then unzip the file):<br>
<code>wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz</code><br>
<code>gunzip UP000005640_9606.fasta.gz</code><br>

2) Build the database as an indexting task needed for an efficient search (<code>makeblastdb</code> is part of the BLAST):<br>
<code>makeblastdb -in UP000005640_9606.fasta -dbtype prot</code><br>

3) Apply <code>blastp</code> by: (<code>-outfmt 6</code> means output be saved in tabular format)<br>
Notice: On Compute Canada servers, for faster computation, use <code>sbatch</code> command with proper settings  (Set <code>--cpus-per-task=16</code> and <code>--mem-per-cpu=512M</code>)<br>
<code>blastp -query UP000005640_9606.fasta -db UP000005640_9606.fasta -evalue 10 -outfmt 6 -num_threads 16 -out human_vs_human_blastp_results.tab</code><br>

4) Zip the result for a faster transfer/loading: (the result of this step, which is **human_vs_human_blastp_results.tab.gz**, is going to be processed in this notebook) <br>
<code>gzip human_vs_human_blastp_results.tab</code><br>

---
#### yeast sequence (FASTA)<br>
<font size="2">
1) Download the protein sequences (yeas) in zipped FASTA format (then unzip the file):<br>
<code>wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000002311/UP000002311_559292.fasta.gz</code><br>
<code>gunzip UP000002311_559292.fasta.gz</code><br>

2) Build the database as an indexting task needed for an efficient search (<code>makeblastdb</code> is part of the BLAST):<br>
<code>makeblastdb -in UP000002311_559292.fasta -dbtype prot</code><br>

3) Apply <code>blastp</code> by: (<code>-outfmt 6</code> means output be saved in tabular format)<br>
Notice: On Compute Canada servers, for faster computation, use <code>sbatch</code> command with proper settings  (Set <code>--cpus-per-task=16</code> and <code>--mem-per-cpu=512M</code>)<br>
<code>blastp -query UP000002311_559292.fasta -db UP000002311_559292.fasta -evalue 10 -outfmt 6 -num_threads 16 -out yeast_vs_yeast_blastp_results.tab</code><br>

4) Zip the results for a faster transfer/loading: (the result of this step, which is **yeast_vs_yeast_blastp_results.tab.gz**, is going to be processed in this notebook)<br>
<code>gzip yeast_vs_yeast_blastp_results.tab</code><br>

</font>

[back to top](#top)<br>

## Import<a id='2'></a>
[back to top](#top)<br>

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import easydict
import linecache
import pprint
import random

pp = pprint.PrettyPrinter(indent=4)

## Configuration<a id='3'></a>
[back to top](#top)<br>

In [ ]:
species = 'yeast' # species of interest to load of and save the resut for

if species=='human':
    association_file_name = 'goa_human.gaf.gz' # human
    blastp_file = 'human_vs_human_blastp_results.tab.gz'
    association_file_url = 'http://geneontology.org/gene-associations/goa_human.gaf.gz'
elif species=='yeast':
    association_file_name = 'sgd.gaf.gz' # yeast
    blastp_file = 'yeast_vs_yeast_blastp_results.tab.gz'
    association_file_url = 'http://current.geneontology.org/annotations/sgd.gaf.gz'
    
args = easydict.EasyDict({
    "go_dir": 'gene_ontology/raw/',    # directory to the Gene Ontology 'go.obo' file
    "association_file_dir": 'species/{}/association_file/raw'.format(species), # directory to the human association file
    "sequence_homology_raw_dir": 'species/{}/sequence_homology/raw'.format(species),   # directory to the raw sequence_homology data computed from BLAST
    "result_sequence_homology_dir": 'species/{}/sequence_homology/processed'.format(species), # directory in which the results would be saved
    #"epectation_value": 0.1,           # the score to filter out low confident interactions (only those above the score are kept)
    "download_gene_ontology": True,    # download the latest version of gene ontology into the specified directory above
    "download_association_file": True  # download association file of the specieis of interest into the specified directory above
})

os.makedirs(args.result_sequence_homology_dir, exist_ok=True)  # create 'result_sequence_homology_dir' folder (if it does not exist already)

subontology_map = {"C":"CC", "P":"BP", "F":"MF"}

In [ ]:
assert os.path.exists(f"{args.sequence_homology_raw_dir}/{blastp_file}") is True, f"\nYou need to prepare the blastp file first based on the guideline provided above! \nPut the {blastp_file} file in '{args.sequence_homology_raw_dir}/' directory."

## Loading Gene Ontology<a id='4'></a>
[back to top](#top)<br>

In [ ]:
if args.download_gene_ontology:
    os.makedirs(args.go_dir, exist_ok=True)  # create 'data_loc' folder (if it does not exist already)
    print("Downloading the latest version of Gene Ontology into '{}'...".format(args.go_dir))
    url = 'http://current.geneontology.org/ontology/go.obo'
    r = requests.get(url, allow_redirects=True)
    open('{}/go.obo'.format(args.go_dir), 'wb').write(r.content)

print("Gene Ontology {}".format(linecache.getline('{}/go.obo'.format(args.go_dir), 2))) # Now: releases/2020-03-23

In [ ]:
"""Reading Gene Ontology to extract Terms and their Descriptive Names"""
with open("{}/go.obo".format(args.go_dir)) as f:
    content = f.readlines()
content = "".join([x for x in content])
content = content.split("[Typedef]")[0].split("[Term]")
print("Information of the last GO term in the file:\n~~~~~~~~~~~~~~~~~~~~~~~~~{}".format(content[-1]))

In [ ]:
"""Going through every GO term and extract information needed ('id', 'alt_id', 'namespace', and 'is_obsolete')"""
go_term_dict = {}
for c in content:
    go_id = ''
    for l in c.split("\n"):
        # id
        if "id: GO:" in l[0:len("id: GO:")]:
            go_id = l.split("id: ")[1]
            go_term_dict[go_id] = {}
        # alt_id
        if "alt_id:" in l[0:len("alt_id")+1]:
            go_term_dict[go_id].setdefault("alt_id", []).append(l.split("alt_id: ")[1])
        # namespace
        if "namespace:" in l[0:len("namespace")+1]:
            go_term_dict[go_id]["namespace"] = l.split("namespace: ")[1]
        # is_obsolete
        if "is_obsolete:" in l[0:len("is_obsolete")+1]:
            go_term_dict[go_id]["is_obsolete"] = l.split("is_obsolete: ")[1]

In [ ]:
"""printing how the key:values are organized for every GO term"""
for i in range(15):
    print(list(go_term_dict)[i], end=": ")
    pp.pprint(go_term_dict[list(go_term_dict)[i]])

In [ ]:
"""grouping GO terms based on the sub-ontologies they belong to"""
subontology_go_term_dict = {}
for go_id in go_term_dict:
    if not go_term_dict[go_id].get('is_obsolete', False): # or => if 'is_obsolete' not in go_term_dict[go_id]:
        subontology_go_term_dict.setdefault(go_term_dict[go_id]['namespace'].split('_')[1][0].upper(), []).append(go_id)

In [ ]:
"""including 'alt_id' into the sub-ontology's groups of GO terms"""
for go_id in go_term_dict:
    if go_term_dict[go_id].get('alt_id', False): # or => if 'alt_id' in go_term_dict[go_id]:
        for alt_id in go_term_dict[go_id].get('alt_id'):
            subontology_go_term_dict[go_term_dict[go_id]['namespace'].split('_')[1][0].upper()].append(alt_id)

In [ ]:
"""printing how the key:values are organized for different sub-ontologies"""
for subontology in subontology_go_term_dict:
    print("{} ({}):: {} <= {} GO term (with 'alt_id') => {}".format(
        subontology, 
        subontology_map[subontology], 
        " ".join(subontology_go_term_dict[subontology][:3]), 
        len(subontology_go_term_dict[subontology]), 
        " ".join(subontology_go_term_dict[subontology][-3:])))

## Loading Genes and Annotations<a id='5'></a>
[back to top](#top)<br>

In [ ]:
if args.download_association_file:
    os.makedirs(args.association_file_dir, exist_ok=True)  # create 'data_loc' folder (if it does not exist already)
    print("Downloading the latest version of association file into '{}'...".format(args.association_file_dir))
    r = requests.get(association_file_url, allow_redirects=True)
    open('{}/{}'.format(args.association_file_dir, association_file_name), 'wb').write(r.content)
print("Done!")

In [ ]:
df = pd.read_csv("{}/{}".format(args.association_file_dir, association_file_name), sep='\t', comment="!", skip_blank_lines=True, header=None, dtype=str)
df = df.iloc[:,[1, 2, 3, 4, 6, 8]]
if len(df[df[3].isnull()])==0:
    df = df[~df[3].str.contains("NOT")]
    df = df.dropna().reset_index(drop=True)
else:
    df = df[df[3].isnull()]
    df = df.dropna().reset_index(drop=True)
df = df.drop(df.columns[2], axis=1)
df

In [ ]:
"""keeping track of the gene ids and their mappings"""
protein_gene_id_map = {}
for gene_id, protein_id in zip(df[1], df[2]):
    protein_gene_id_map[protein_id] = gene_id

##### removing 'ND' and 'IEA' annotations

In [ ]:
df = df[(df[6]!='ND') & (df[6]!='IEA')]
df

In [ ]:
"""protein dictionary to keep track of annotations for proteins (from each sub-ontology)"""
proteins_dict = {}
for index, row in df.iterrows():
    gene = row[1]
    go_term_id = row[4]
    subontology = row[8]
    if go_term_id in subontology_go_term_dict[subontology]:
        proteins_dict.setdefault(gene, dict()).setdefault(subontology, set()).add(go_term_id)
        
"""printing how the key:values are organized for every gene/protein"""
for i in range(5):
    print(list(proteins_dict)[i], end=": ")
    pp.pprint(proteins_dict[list(proteins_dict)[i]])
print("\nTotal number of genes/proteins annotated:", len(proteins_dict))

#### Taking into account only fully annotated genes/proteins

In [ ]:
"""keeping track of fully annotated genes/proteins"""
fully_annotated_proteins_wo_iea = []
for protein in proteins_dict:
    if len(proteins_dict[protein]) == 3:
        fully_annotated_proteins_wo_iea.append(protein)
print("Out of {} proteins {} are (experimentally or manually) annotated by all three sub-ontologies.".format(len(proteins_dict), len(fully_annotated_proteins_wo_iea)))

## Loading Sequence Homology data<a id='6'></a>
[back to top](#top)<br>

#### Loading species Sequence Homology data

In [ ]:
# column names of the result tabular file from blastp
column_header = ['qseqid',   # query (e.g., unknown gene) sequence id
                 'sseqid',   # subject (e.g., reference genome) sequence id
                 'pident',   # percentage of identical matches
                 'length',   # alignment length (sequence overlap)
                 'mismatch', # number of mismatches
                 'gapopen',  # number of gap openings
                 'qstart',   # start of alignment in query
                 'qend',     # end of alignment in query
                 'sstart',   # start of alignment in subject
                 'send',     # end of alignment in subject
                 'evalue',   # expect value
                 'bitscore'] # bit score

In [ ]:
df_sh = pd.read_csv("{}/{}".format(args.sequence_homology_raw_dir, blastp_file),
                     names=column_header , sep='\t', header=None)
df_sh

In [ ]:
if species=='yeast':
    df_sh['qseqid'] = df_sh.qseqid.str.split("|", expand = True)[2]
    df_sh['sseqid'] = df_sh.sseqid.str.split("|", expand = True)[2]
elif species=='human':
    df_sh['qseqid'] = df_sh.qseqid.str.split("|", expand = True)[1]
    df_sh['sseqid'] = df_sh.sseqid.str.split("|", expand = True)[1]
df_sh

In [ ]:
df_sh = df_sh.iloc[:,[0, 1, 10, 11]]
#df_sh = df_sh[df_sh['evalue'] < args.epectation_value]
df_sh

#### ID Mapping from "UniProtKB AC/ID" to "Gene name" (only for yeast)
For mor information refer to :[https://www.uniprot.org/help/api_idmapping](https://www.uniprot.org/help/api_idmapping)

In [ ]:
if species=='yeast':
    """first, collecting all 'UniProtKB AC/ID' ids"""
    ACC_ID_ids = " ".join(sorted(list(set(df_sh['qseqid']) | set(df_sh['sseqid']))))

    import urllib.parse
    import urllib.request

    url = 'https://www.uniprot.org/uploadlists/'

    params = {
    'from': 'ACC+ID',
    'to': 'SGD_ID',
    'format': 'tab',
    'query': ACC_ID_ids
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url, data)
    with urllib.request.urlopen(req) as f:
        response = f.read()

    GENENAME_ids = {} # Gene name ids as values
    for i, mapping in enumerate(response.decode('utf-8').strip().split("\n")):
        if i!=0: 
            id1, id2 = mapping.split("\t")
            GENENAME_ids[id1] = id2

#### Collecting fully annotated protein pairs with their bitscores

In [ ]:
sequence_homology_dict = {}
if species=='yeast':
    for protein1, protein2, bitscore in zip(df_sh['qseqid'] , df_sh['sseqid'], df_sh['bitscore']):
        if (protein1 in GENENAME_ids) and (protein2 in GENENAME_ids):
            pr1, pr2 = GENENAME_ids[protein1], GENENAME_ids[protein2]
            if (pr1 in fully_annotated_proteins_wo_iea) and (pr2 in fully_annotated_proteins_wo_iea):
                sequence_homology_dict[(pr1, pr2)] = bitscore
if species=='human':
    for protein1, protein2, bitscore in zip(df_sh['qseqid'] , df_sh['sseqid'], df_sh['bitscore']):
        if (protein1 in fully_annotated_proteins_wo_iea) and (protein2 in fully_annotated_proteins_wo_iea):
            sequence_homology_dict[(protein1, protein2)] = bitscore

print("Number of fully annotated protein pairs is (non-symetric): {}".format(len(sequence_homology_dict)))

## Computing LRBS and RRBS scores<a id='7'></a>

LRBS = log-reciprocal BLAST score<br>
RRBS = relative reciprocal BLAST score<br>

[back to top](#top)<br>

In [ ]:
def lrbs(protein1, protein2):
    """function that computes and returns the log-reciprocal BLAST score between input genes"""
    numerator = sequence_homology_dict[(protein1, protein2)] + sequence_homology_dict[(protein2, protein1)]
    return np.log10(numerator/2)

def rrbs(protein1, gene2):
    """function that computes and returns the relative reciprocal BLAST score between input genes"""
    numerator = sequence_homology_dict[(protein1, protein2)] + sequence_homology_dict[(protein2, protein1)]
    denominator = sequence_homology_dict[(protein1, protein1)] + sequence_homology_dict[(protein2, protein2)]
    return numerator/denominator

In [ ]:
lrbs_rrbs_results = {}
for protein1, protein2 in sequence_homology_dict:
    if (protein2, protein1) in sequence_homology_dict and protein1!=protein2 and (protein2, protein1) not in lrbs_rrbs_results:
        lrbs_rrbs_results[(protein1, protein2)] = {'lrbs': lrbs(protein1, protein2), 'rrbs': rrbs(protein1, protein2)}

print("Number of protein pairs for sequence homology experiment ({}): {}".format(species, len(lrbs_rrbs_results)))

## Saving the results<a id='8'></a>
[back to top](#top)<br>

In [ ]:
with open(f'{args.result_sequence_homology_dir}/{species}_sequence_homology.tsv', 'w') as fw:
    fw.write("Protein_1\tProtein_2\tLRBS\tRRBS\n")
    for protein1, protein2 in sorted(lrbs_rrbs_results):
        fw.write("{}\t{}\t{}\t{}\n".format(protein1, protein2, 
                                        lrbs_rrbs_results[(protein1, protein2)]['lrbs'],
                                        lrbs_rrbs_results[(protein1, protein2)]['rrbs']
                                       ))

[back to top](#top)<br>

In [ ]:
df = pd.read_csv(f"{args.result_sequence_homology_dir}/{species}_sequence_homology.tsv", sep="\t", dtype=str)
df

In [ ]:
sh_genes = set(list(df.Protein_1) + list(df.Protein_2))
print(f"Number of {species} genes:", len(sh_genes))
with open(f'{args.result_sequence_homology_dir}/{species}_sequence_homology_genes.tsv', 'w') as fw:
    for gene in sorted(sh_genes):
        fw.write(f"{gene}\n")

[back to top](#top)<br>

---